In [1]:
#FIRST HALF OF THIS CODE COPIED FROM LESSON 1
# these allow you to modify source code of imports, notebook automatically updates
%load_ext autoreload
%autoreload 2
# show plots in notebook
%matplotlib inline 

In [6]:
from fastai.fastai.imports import *
from fastai.old.fastai.structured import *

# from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor
from IPython.display import display

from sklearn import metrics

In [145]:
PATH1 = 'data/BAYZF.csv'
PATH2 = 'data/HON.csv'
PATH3 = 'data/MMM.csv'
PATH4 = 'data/SYF.csv'
dateColumn = "Date"
df_BAYZF = pd.read_csv(PATH1, low_memory=False, parse_dates=[dateColumn])
df_HON = pd.read_csv(PATH2, low_memory=False, parse_dates=[dateColumn])
df_MMM = pd.read_csv(PATH3, low_memory=False, parse_dates=[dateColumn])
df_SYF = pd.read_csv(PATH4, low_memory=False, parse_dates=[dateColumn])

In [110]:
df_BAYZF.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2009-02-13,55.549999,55.549999,55.549999,55.549999,44.911427,100.0
1,2009-02-17,53.400002,53.400002,53.099998,53.200001,43.011486,400.0
2,2009-02-18,53.400002,53.400002,53.299999,53.299999,43.092339,300.0
3,2009-02-19,54.250000,54.250000,53.750000,53.750000,43.456146,500.0
4,2009-02-20,52.299999,52.299999,51.900002,51.910000,41.968533,500.0


In [20]:
df_HON.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2009-01-02,31.394459,33.224930,31.003576,33.043789,25.723433,5360700.0
1,2009-01-05,32.862648,33.062855,32.414562,32.872181,25.589838,7326500.0
2,2009-01-06,33.196327,34.702652,33.196327,34.359436,26.747620,7553800.0
3,2009-01-07,33.959023,33.959023,32.586170,32.843578,25.567570,5969900.0
4,2009-01-08,32.624306,32.986584,32.223888,32.986584,25.678898,4573200.0


In [21]:
df_MMM.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2009-01-02,57.549999,59.389999,57.520000,59.189999,45.690918,5313900.0
1,2009-01-05,58.790001,59.090000,58.110001,58.500000,45.158264,3768800.0
2,2009-01-06,58.740002,59.810001,58.610001,59.189999,45.690918,4966200.0
3,2009-01-07,58.610001,58.959999,57.580002,58.070000,44.826347,4598100.0
4,2009-01-08,57.680000,58.709999,57.450001,58.580002,45.220032,3944900.0


In [22]:
df_SYF.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-07-31,23.000000,24.000000,22.60,23.00,21.73275,56483900.0
1,2014-08-01,22.900000,23.000000,22.67,23.00,21.73275,6458900.0
2,2014-08-04,23.100000,23.129999,22.90,23.00,21.73275,3455900.0
3,2014-08-05,22.990000,23.010000,22.67,23.00,21.73275,4450200.0
4,2014-08-06,22.799999,22.980000,22.67,22.98,21.71385,6993400.0


In [146]:
df_raw = df_BAYZF

In [147]:
add_datepart(df_raw, dateColumn)

In [ ]:
# makes all categorical variables get an associated number, although data looks same if printed
# train_cats(df_raw)

In [148]:
# goes through each column, find the nulls, adds em, divides by the length
# kind of op syntax here
# df_raw.isnull returns a new dataframe with true for null and false for non-null from original df
df_raw.isnull().sum().sort_index()/len(df_raw)

Adj Close           0.009952
Close               0.009952
Day                 0.009952
Dayofweek           0.009952
Dayofyear           0.009952
Elapsed             0.000000
High                0.009952
Is_month_end        0.000000
Is_month_start      0.000000
Is_quarter_end      0.000000
Is_quarter_start    0.000000
Is_year_end         0.000000
Is_year_start       0.000000
Low                 0.009952
Month               0.009952
Open                0.009952
Volume              0.009952
Week                0.009952
Year                0.009952
dtype: float64

In [96]:
?proc_df

In [149]:
to_predict = "Close"
# options: ["Adj Close", "Open", "High", "Low", "Volume"]
to_drop = ["Adj Close", "High", "Low", "Volume"]
result = proc_df(df_raw, y_fld=to_predict, skip_flds=to_drop)
df = result[0]
y = result[1]

In [150]:
to_remove = np.argwhere(np.isnan(y))
to_remove = [row[0] for row in to_remove.tolist()]
print(to_remove)
df = df.drop(to_remove, axis=0)
y = y[~np.isnan(y)]

[2487, 2488, 2489, 2490, 2491, 2492, 2493, 2494, 2495, 2496, 2497, 2498, 2499, 2500, 2501, 2502, 2503, 2504, 2505, 2506, 2507, 2508, 2509, 2510, 2511]


In [130]:
print(y)

[55.55 53.2  53.3  ... 67.46 68.87 69.5 ]


In [131]:
df

,Open,High,Low,Volume,Year,Month,Week,Day,Dayofweek,Dayofyear,...,Open_na,High_na,Low_na,Volume_na,Year_na,Month_na,Week_na,Day_na,Dayofweek_na,Dayofyear_na
0,55.549999,55.549999,55.549999,100.0,2009.0,2.0,7.0,13.0,4.0,44.0,...,False,False,False,False,False,False,False,False,False,False
1,53.400002,53.400002,53.099998,400.0,2009.0,2.0,8.0,17.0,1.0,48.0,...,False,False,False,False,False,False,False,False,False,False
2,53.400002,53.400002,53.299999,300.0,2009.0,2.0,8.0,18.0,2.0,49.0,...,False,False,False,False,False,False,False,False,False,False
3,54.250000,54.250000,53.750000,500.0,2009.0,2.0,8.0,19.0,3.0,50.0,...,False,False,False,False,False,False,False,False,False,False
4,52.299999,52.299999,51.900002,500.0,2009.0,2.0,8.0,20.0,4.0,51.0,...,False,False,False,False,False,False,False,False,False,False
5,53.540001,53.540001,51.750000,1100.0,2009.0,2.0,9.0,23.0,0.0,54.0,...,False,False,False,False,False,False,False,False,False,False
6,52.500000,52.500000,51.250000,2500.0,2009.0,2.0,9.0,24.0,1.0,55.0,...,False,False,False,False,False,False,False,False,False,False
7,50.660000,50.660000,50.660000,100.0,2009.0,2.0,9.0,25.0,2.0,56.0,...,False,False,False,False,False,False,False,False,False,False
8,48.500000,48.500000,48.500000,100.0,2009.0,2.0,9.0,26.0,3.0,57.0,...,False,False,False,False,False,False,False,False,False,False
9,47.700001,48.650002,47.700001,1700.0,2009.0,2.0,9.0,27.0,4.0,58.0,...,False,False,False,False,False,False,False,False,False,False


In [132]:
print(len(y))

2487


In [151]:
# splits dataset based on how many you want to train/validate
def split_vals(a, n):
    return a[:n].copy(), a[n:].copy()

n_valid = 365 # for the final year
n_train = len(df) - n_valid
# raw just uses the df_raw
raw_train, raw_valid = split_vals(df_raw, n_train)
X_train, X_valid = split_vals(df, n_train)
y_train, y_valid = split_vals(y, n_train)

In [152]:
X_train

,Open,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,...,Is_year_end,Is_year_start,Elapsed,Open_na,Year_na,Month_na,Week_na,Day_na,Dayofweek_na,Dayofyear_na
0,55.549999,2009.0,2.0,7.0,13.0,4.0,44.0,False,False,False,...,False,False,1234483200,False,False,False,False,False,False,False
1,53.400002,2009.0,2.0,8.0,17.0,1.0,48.0,False,False,False,...,False,False,1234828800,False,False,False,False,False,False,False
2,53.400002,2009.0,2.0,8.0,18.0,2.0,49.0,False,False,False,...,False,False,1234915200,False,False,False,False,False,False,False
3,54.250000,2009.0,2.0,8.0,19.0,3.0,50.0,False,False,False,...,False,False,1235001600,False,False,False,False,False,False,False
4,52.299999,2009.0,2.0,8.0,20.0,4.0,51.0,False,False,False,...,False,False,1235088000,False,False,False,False,False,False,False
5,53.540001,2009.0,2.0,9.0,23.0,0.0,54.0,False,False,False,...,False,False,1235347200,False,False,False,False,False,False,False
6,52.500000,2009.0,2.0,9.0,24.0,1.0,55.0,False,False,False,...,False,False,1235433600,False,False,False,False,False,False,False
7,50.660000,2009.0,2.0,9.0,25.0,2.0,56.0,False,False,False,...,False,False,1235520000,False,False,False,False,False,False,False
8,48.500000,2009.0,2.0,9.0,26.0,3.0,57.0,False,False,False,...,False,False,1235606400,False,False,False,False,False,False,False
9,47.700001,2009.0,2.0,9.0,27.0,4.0,58.0,False,False,False,...,False,False,1235692800,False,False,False,False,False,False,False


In [135]:
y_train

array([ 55.55   ,  53.2    ,  53.3    , ..., 128.36999, 128.36999, 128.36999])

In [136]:
X_valid

,Open,High,Low,Volume,Year,Month,Week,Day,Dayofweek,Dayofyear,...,Open_na,High_na,Low_na,Volume_na,Year_na,Month_na,Week_na,Day_na,Dayofweek_na,Dayofyear_na
2122,128.369995,128.369995,128.369995,0.0,2017.0,7.0,29.0,20.0,3.0,201.0,...,False,False,False,False,False,False,False,False,False,False
2123,128.369995,128.369995,128.369995,0.0,2017.0,7.0,29.0,21.0,4.0,202.0,...,False,False,False,False,False,False,False,False,False,False
2124,128.149994,128.149994,128.149994,200.0,2017.0,7.0,30.0,24.0,0.0,205.0,...,False,False,False,False,False,False,False,False,False,False
2125,130.479996,131.050003,128.699997,2300.0,2017.0,7.0,30.0,25.0,1.0,206.0,...,False,False,False,False,False,False,False,False,False,False
2126,128.649994,130.949997,128.649994,7700.0,2017.0,7.0,30.0,26.0,2.0,207.0,...,False,False,False,False,False,False,False,False,False,False
2127,130.949997,130.949997,130.949997,0.0,2017.0,7.0,30.0,27.0,3.0,208.0,...,False,False,False,False,False,False,False,False,False,False
2128,126.949997,126.949997,125.169998,29600.0,2017.0,7.0,30.0,28.0,4.0,209.0,...,False,False,False,False,False,False,False,False,False,False
2129,126.889999,126.889999,126.889999,0.0,2017.0,7.0,31.0,31.0,0.0,212.0,...,False,False,False,False,False,False,False,False,False,False
2130,126.400002,126.800003,126.400002,900.0,2017.0,8.0,31.0,1.0,1.0,213.0,...,False,False,False,False,False,False,False,False,False,False
2131,126.699997,126.699997,125.500000,700.0,2017.0,8.0,31.0,2.0,2.0,214.0,...,False,False,False,False,False,False,False,False,False,False


In [137]:
y_valid

array([128.36999, 128.36999, 128.14999, 128.7    , 130.95   , 130.95   , 126.89   , 126.89   , 126.5    ,
       125.5    , 126.9    , 126.06   , 126.06   , 124.     , 123.79   , 123.79   , 124.06   , 126.25   ,
       125.55   , 125.65   , 125.65   , 126.     , 126.     , 126.     , 126.98   , 126.05   , 129.89999,
       129.89999, 129.89999, 129.89999, 129.89999, 129.89999, 129.2    , 129.10001, 132.26   , 131.5    ,
       131.5    , 132.25   , 132.25   , 132.25   , 132.25   , 132.25   , 132.25   , 132.25   , 132.     ,
       132.     , 131.     , 131.75   , 130.14999, 130.14999, 137.5    , 133.2    , 133.2    , 133.2    ,
       133.2    , 138.39999, 138.39999, 138.39999, 138.     , 138.     , 138.     , 139.75   , 137.98   ,
       138.55   , 138.55   , 138.55   , 138.55   , 138.55   , 135.82001, 135.82001, 135.82001, 130.48   ,
       130.48   , 131.7    , 131.7    , 133.3    , 134.     , 132.71001, 132.71001, 127.45   , 126.97   ,
       126.97   , 126.97   , 126.97   , 126.97

In [153]:
def rmse(x, y):
    return math.sqrt(((x-y)**2).mean())

# function for print r2 and RMSE of training data and validation data
def print_score(rf):
    res = [rmse(rf.predict(X_train), y_train), rmse(rf.predict(X_valid), y_valid), rf.score(X_train, y_train), rf.score(X_valid, y_valid)]
    print(res)

In [154]:
rf = RandomForestRegressor(n_estimators=40, min_samples_leaf=3, max_features=0.5, n_jobs=-1)
rf.fit(X_train, y_train)
print_score(rf)

[0.48039875907108487, 7.9519712294890015, 0.9997507968436206, 0.840983665395963]


In [155]:
??rf_feat_importance

In [156]:
fi = rf_feat_importance(rf, df)
fi[:10]

,cols,imp
0,Open,0.577960
13,Elapsed,0.220627
1,Year,0.192462
6,Dayofyear,0.004913
3,Week,0.002758
2,Month,0.000839
4,Day,0.000344
5,Dayofweek,0.000087
8,Is_month_start,0.000005
7,Is_month_end,0.000003
